[//]: # ( Plant Card Generator based on public and free API queries. )
[//]: # ( Notebook language is Python 3 and uses requests and PIL libraries. )
[//]: # ( This notebook is designed to create plant care cards by fetching data from various APIs. )
[//]: # ( The generated cards will include information such as plant name, care instructions, and images.)
[//]: # ( # License: MIT License )
[//]: # ( Date: 2024-06-15 )
[//]: # ( Creator: Pekka Sihvonen )
[//]: # ( Language: Finnish )

# 🌲 Kasvikortti Generaattori

Tällä generaattorilla haet kasvin tieteelliset tiedot tietokannoista, joista osa vaatii API avaimen. Talleta API avaimet muistikirjaan käyttämällä **Muuttujat** (Variables) tai **Salaisuudet** (Secrets) toimintoa.

## ⚙️ Asetukset

Lisää tarvittavat API-avaimet muistikirjan muuttujiin ja valitse käytettävät tietolähteet alla olevasta käyttöliittymästä. Paina lopuksi **Tallenna asetukset** ennen hakufunktioiden suorittamista.

Alla linkkejä API palveluihin:

[Gemini API](https://aistudio.google.com)

[Laji API](https://api.laji.fi)

[Trefle API](https://trefle.com)

[BHL API](https://www.biodiversitylibrary.org/api2/docs/)

### Muut tietolähteet
- **Wikipedia REST API** — tiivistetty yleistietoa ja mahdollinen kuva/artikkelilinkki. (HUOM! ei tieteellisesti hyväksyttyä tietoa)
- **iNaturalist API** — yhteisön havaintomäärä, yleisnimi ja oletuskuva.
- **GBIF distributions -rajapinta** — levinneisyysalueiden tiivistelmät taksoniavainperusteisesti.

### Konfigurointi paneeli
Aja seuraava koodi, joka asentaa tarvittavat Python -ohjelmointikirjastot muistikirjaan. Täytä sen jälkeen alla olevat kentät ja valitse käytettävät tietolähteet. Paina lopuksi **Tallenna asetukset** ennen sitä seuraavien hakufunktioiden suorittamista.

In [ ]:
# 1.1 Asennus ja kirjastojen tuonti
!pip install requests pandas google-generativeai
from IPython.display import display, Markdown, Image
import requests
import pandas as pd
import ipywidgets as widgets
import google.generativeai as genai
from google.colab import userdata # Used to securely store your API key

# 1.2 Käyttöliittymä syötteille
kasvin_nimi_widget = widgets.Text(
    value=globals().get("kasvin_nimi", ""),
    placeholder="Esim. Quercus robur",
    description="Kasvi",
    disabled=False,
)

asetukset_oletus = globals().get("API_ASETUKSET", {})

use_gbif_widget = widgets.Checkbox(
    value=asetukset_oletus.get("gbif", True),
    description="Hae GBIF taksonomia",
)

use_trefle_widget = widgets.Checkbox(
    value=asetukset_oletus.get("trefle", True),
    description="Hae Trefle tiedot ja kuva",
)

use_laji_widget = widgets.Checkbox(
    value=asetukset_oletus.get("laji", True),
    description="Hae Laji.fi suomi-nimi",
)

use_eol_widget = widgets.Checkbox(
    value=asetukset_oletus.get("eol", True),
    description="Hae EOL ekologiset tiedot",
)

use_gbif_occ_widget = widgets.Checkbox(
    value=asetukset_oletus.get("gbif_occ", True),
    description="Hae GBIF esiintymismäärä",
)

use_gbif_dist_widget = widgets.Checkbox(
    value=asetukset_oletus.get("gbif_dist", True),
    description="Hae GBIF levinneisyysalueet",
)

use_wikipedia_widget = widgets.Checkbox(
    value=asetukset_oletus.get("wikipedia", True),
    description="Hae Wikipedia tiivistelmä",
)

use_inaturalist_widget = widgets.Checkbox(
    value=asetukset_oletus.get("inaturalist", True),
    description="Hae iNaturalist havaintotiedot",
)

use_wikimedia_widget = widgets.Checkbox(
    value=asetukset_oletus.get("wikimedia", True),
    description="Hae Wikimedia kasvikuvitus",
)

use_bhl_widget = widgets.Checkbox(
    value=asetukset_oletus.get("bhl", False),
    description="Hae BHL kuvitus (vaatii API-avaimen)",
)

use_gemini_widget = widgets.Checkbox(
    value=asetukset_oletus.get("gemini", True),
    description="Generoi kuvaus Geminillä",
)

tallenna_button = widgets.Button(
    description="Tallenna asetukset",
    button_style="success",
    icon="check",
)

status_output = widgets.Output()

def paivita_asetukset(_=None):
    global kasvin_nimi, TREFLE_API_KEY, LAJI_TOKEN, GEMINI_API_KEY, BHL_API_KEY, API_ASETUKSET
    kasvin_nimi = kasvin_nimi_widget.value.strip()
    # Fetch API keys from userdata
    try:
        TREFLE_API_KEY = userdata.get('TREFLE_API_KEY')
    except userdata.SecretNotFoundError:
        TREFLE_API_KEY = ''
        print("TREFLE_API_KEY not found in secrets.")

    try:
        LAJI_TOKEN = userdata.get('LAJI_TOKEN')
    except userdata.SecretNotFoundError:
        LAJI_TOKEN = ''
        print("LAJI_TOKEN not found in secrets.")

    try:
        GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY') # Using GOOGLE_API_KEY for Gemini
    except userdata.SecretNotFoundError:
        GEMINI_API_KEY = ''
        print("GOOGLE_API_KEY not found in secrets for Gemini.")

    try:
        BHL_API_KEY = userdata.get('BHL_API_KEY')
    except userdata.SecretNotFoundError:
        BHL_API_KEY = ''
        print("BHL_API_KEY not found in secrets.")


    API_ASETUKSET = {
        "gbif": use_gbif_widget.value,
        "trefle": use_trefle_widget.value,
        "laji": use_laji_widget.value,
        "eol": use_eol_widget.value,
        "gbif_occ": use_gbif_occ_widget.value,
        "gbif_dist": use_gbif_dist_widget.value,
        "wikipedia": use_wikipedia_widget.value,
        "inaturalist": use_inaturalist_widget.value,
        "wikimedia": use_wikimedia_widget.value,
        "bhl": use_bhl_widget.value,
        "gemini": use_gemini_widget.value,
    }
    with status_output:
        status_output.clear_output()
        if not kasvin_nimi:
            print("⚠️ Syötä kasvin tieteellinen nimi.")
        else:
            print("✅ Asetukset päivitetty.")
            # Optional: Print status of API keys
            print(f"Trefle API Key: {'Asetettu' if TREFLE_API_KEY else 'Puuttuu'}")
            print(f"Laji.fi Token: {'Asetettu' if LAJI_TOKEN else 'Puuttuu'}")
            print(f"Gemini API Key: {'Asetettu' if GEMINI_API_KEY else 'Puuttuu'}")
            print(f"BHL API Key: {'Asetettu' if BHL_API_KEY else 'Puuttuu'}")


tallenna_button.on_click(paivita_asetukset)

# Initialize settings on load
paivita_asetukset()

paneeli = widgets.VBox([
    widgets.HTML("<b>Kasvin tiedot</b>"),
    kasvin_nimi_widget,
    widgets.HTML("<b>Tietolähteet</b>"),
    widgets.GridBox([
        use_gbif_widget, use_trefle_widget, use_laji_widget, use_wikimedia_widget,
        use_eol_widget, use_gbif_occ_widget, use_gbif_dist_widget, use_wikipedia_widget,
        use_inaturalist_widget, use_bhl_widget, use_gemini_widget
    ], layout=widgets.Layout(grid_template_columns="repeat(3, 33%)")),
    widgets.HBox([tallenna_button, status_output])
])

display(paneeli)

In [ ]:
# 1.2 Asetusten tarkistus
def tarkista_asetukset():
    if not kasvin_nimi:
        raise ValueError("Syötä kasvin tieteellinen nimi käyttöliittymässä ja paina 'Tallenna asetukset'.")
    print(f"Kasvi: {kasvin_nimi}")
    print(f"Käytettävät tietolähteet: {', '.join([nimi for nimi, arvo in API_ASETUKSET.items() if arvo])}")
    if API_ASETUKSET.get("trefle") and not TREFLE_API_KEY:
        print("⚠️ Trefle API key puuttuu. Trefle-haut ohitetaan.")
    if API_ASETUKSET.get("laji") and not LAJI_TOKEN:
        print("⚠️ Laji.fi token puuttuu. Laji.fi haut ohitetaan.")
    if API_ASETUKSET.get("gemini") and not GEMINI_API_KEY:
        print("⚠️ Gemini API key puuttuu. AI-kuvaus ohitetaan.")
    if API_ASETUKSET.get("bhl") and not BHL_API_KEY:
        print("⚠️ BHL API key puuttuu. BHL-kuvat ohitetaan.")

tarkista_asetukset()

In [ ]:
# 1.3 GBIF haut
 
API_ASETUKSET = globals().get("API_ASETUKSET", {})

def hae_gbif_data(tieteellinen_nimi):
    """Hakee perus taksonomian GBIF API:sta."""
    url = f"https://api.gbif.org/v1/species/match?name={tieteellinen_nimi}"
    response = requests.get(url).json()

    if response.get('confidence') and response['confidence'] >= 90 and response.get('rank') == 'SPECIES':
        data = {
            "Tieteellinen nimi": response.get('scientificName'),
            "Taksonin tila": response.get('taxonomicStatus'),
            "Valtakunta (Kingdom)": response.get('kingdom'),
            "Heimo (Family)": response.get('family'),
            "Suku (Genus)": response.get('genus'),
        }
        print("✅ GBIF: Taksonomia haettu.")
        return data
    else:
        print(f"❌ GBIF ei löytänyt luotettavaa osumaa nimelle: {tieteellinen_nimi}")
        return None

if API_ASETUKSET.get("gbif", True):
    gbif_data = hae_gbif_data(kasvin_nimi)
else:
    print("ℹ️ GBIF taksonomia jätettiin välistä asetuksien perusteella.")
    gbif_data = None

✅ GBIF: Taksonomia haettu.


In [ ]:
API_ASETUKSET = globals().get("API_ASETUKSET", {})

def hae_trefle_data(tieteellinen_nimi, api_key):
    """Hakee yksityiskohtaisia ominaisuuksia ja kuvan Trefle API:sta."""
    if not api_key or api_key == "LIITÄ TÄHÄN OMA TREFLE API-AVAINSI":
        print("❌ Trefle: API-avain puuttuu. Ohitetaan haku.")
        return None

    # Haku tieteellisen nimen perusteella
    search_url = f"https://trefle.io/api/v1/plants/search?token={api_key}&q={tieteellinen_nimi}"
    response = requests.get(search_url).json()

    if response['data']:
        plant_details = response['data'][0]

        # Hakee yksityiskohtaisempaa tietoa toisella kutsulla käyttämällä slug-tunnusta
        slug = plant_details.get('slug')
        details_url = f"https://trefle.io/api/v1/plants/{slug}?token={api_key}"
        details_response = requests.get(details_url).json()

        trefle_data = {
            "Kuva URL (Trefle)": plant_details.get('image_url'),
            "Yleisnimi (English)": plant_details.get('common_name'),
            "Kasvumuoto": details_response.get('data', {}).get('main_species', {}).get('growth_form'),
            "pH-alue": f"{details_response.get('data', {}).get('main_species', {}).get('ph_minimum')} - {details_response.get('data', {}).get('main_species', {}).get('ph_maximum')}"
        }
        print("✅ Trefle: Kuva ja ominaisuudet haettu.")
        return trefle_data

    print("❌ Trefle: Ei löytänyt tietoa.")
    return None

if API_ASETUKSET.get("trefle", True):
    trefle_data = hae_trefle_data(kasvin_nimi, TREFLE_API_KEY)
else:
    print("ℹ️ Trefle-haku jätettiin välistä asetuksien perusteella.")
    trefle_data = None

✅ Trefle: Kuva ja ominaisuudet haettu.


### Laji.fi Suomenkielisen nimen haku

In [ ]:
API_ASETUKSET = globals().get("API_ASETUKSET", {})

def hae_laji_fi_nimi(tieteellinen_nimi):
    """Hakee suomenkielisen nimen Laji.fi:n Taksonomia-API:sta."""
    # Haun toteutus riippuu API-versioista, mutta /taxa/search on yleinen tapa
    url = "https://api.laji.fi/v0/taxa/search"
    params = {
        "q": tieteellinen_nimi,
        "lang": "fi",
        "access_token": LAJI_TOKEN,
        "limit": 1
    }
    # Jos Laji.fi token on virheellinen/puuttuu, tämä voi palauttaa virheen.
    try:
        response = requests.get(url, params=params).json()

        if response and response.get('results'):
            result = response['results'][0]
            # Tarkista, että tieteellinen nimi vastaa riittävän hyvin
            if tieteellinen_nimi.lower() == result.get('scientificName', '').lower():
                 print("✅ Laji.fi: Suomenkielinen nimi haettu.")
                 return result.get('vernacularName', '')

        print("❌ Laji.fi: Suomenkielistä nimeä ei löytynyt tai token on puutteellinen/virheellinen.")
        return "N/A"
    except Exception as e:
        print(f"❌ Laji.fi haku epäonnistui: {e}")
        return "N/A"

if API_ASETUKSET.get("laji", True) and LAJI_TOKEN:
    laji_nimi = hae_laji_fi_nimi(kasvin_nimi)
elif API_ASETUKSET.get("laji", True):
    print("ℹ️ Laji.fi token puuttuu, suomenkielistä nimeä ei haeta.")
    laji_nimi = "N/A"
else:
    print("ℹ️ Laji.fi haku jätettiin välistä asetuksien perusteella.")
    laji_nimi = "N/A"

### Wikimedia Commons kasvitaulu (Botanical Illustration)

In [ ]:
API_ASETUKSET = globals().get("API_ASETUKSET", {})

def hae_kasvitaulu(tieteellinen_nimi):
    """Yrittää hakea kasvitaulun Wikimedia Commonsin API:sta monipuolisilla hauilla."""
    base_url = "https://commons.wikimedia.org/w/api.php"
    search_queries = [
        f'intitle:"{tieteellinen_nimi}" insource:"Botanical illustration"',
        f'intitle:"{tieteellinen_nimi}" insource:"Chromolithograph"',
        f'"{tieteellinen_nimi}" insource:"botanical illustration"',
        f'"{tieteellinen_nimi}" filetype:bitmap',
    ]
    fallback_image = None

    for query in search_queries:
        params = {
            "action": "query",
            "format": "json",
            "prop": "imageinfo",
            "generator": "search",
            "gsrsearch": query,
            "gsrnamespace": "6",
            "gsrlimit": 5,
            "iiprop": "url|mime|extmetadata",
            "iiurlwidth": 1200,
            "origin": "*",
        }

        try:
            response = requests.get(base_url, params=params, timeout=15)
            response.raise_for_status()
            data = response.json()
        except Exception as error:
            print(f"❌ Wikimedia Commons: haku epäonnistui ({error}).")
            continue

        pages = data.get("query", {}).get("pages", {})
        if not pages:
            continue

        for page in pages.values():
            for image_info in page.get("imageinfo", []):
                if not image_info.get("mime", "").startswith("image/"):
                    continue

                image_url = image_info.get("thumburl") or image_info.get("url")
                if not image_url:
                    continue

                metadata = image_info.get("extmetadata", {})
                categories = metadata.get("Categories", {}).get("value", "").lower()
                description = metadata.get("ImageDescription", {}).get("value", "").lower()

                if any(keyword in categories for keyword in ["botanical illustration", "botany", "herbarium"]) or "illustration" in description:
                    print(f"✅ Wikimedia Commons: Kasvitaulu haettu haulla '{query}'.")
                    return image_url

                if fallback_image is None:
                    fallback_image = image_url

    if fallback_image:
        print("ℹ️ Wikimedia Commons: Löydettiin kuva, mutta ei vahvistettua kasvitaulua. Palautetaan paras osuma.")
        return fallback_image

    print(f"❌ Wikimedia Commons: Kasvitaulua ei löytynyt nimellä {tieteellinen_nimi}.")
    return None

if API_ASETUKSET.get("wikimedia", True):
    kasvitaulu_url = hae_kasvitaulu(kasvin_nimi)
else:
    print("ℹ️ Wikimedia Commons -haku jätettiin välistä asetuksien perusteella.")
    kasvitaulu_url = None

### Asetusten tarkastus ja tiedon haku useasta lähteestä

In [27]:
API_ASETUKSET = globals().get("API_ASETUKSET", {})

def hae_eol_data(tieteellinen_nimi):
    """Hakee ekologisia tietoja EOL API:sta."""
    search_url = "http://eol.org/api/search/1.0.json"
    search_params = {
        "q": tieteellinen_nimi,
        "page": 1,
        "exact": True,
        "filter_by_taxon_concept_id": "",
        "filter_by_hierarchy_entry_id": "",
        "filter_by_string": "",
        "cache_ttl": 60,
        "key": "" # EOL API often works without a key for basic searches
    }

    try:
        search_response = requests.get(search_url, params=search_params).json()

        if search_response and search_response.get('results'):
            # Assuming the first result is the correct one
            eol_id = search_response['results'][0]['id']

            # Now get the pages data for more details
            page_url = f"http://eol.org/api/pages/1.0/{eol_id}.json"
            page_params = {
                "cache_ttl": 60,
                "details": True,
                "common_names": True,
                "synonyms": True,
                "references": False,
                "vetted": 0,
                "subject": "Habitat", # Request Habitat information specifically
                "subject": "Reproduction", # Request Reproduction information
                "key": ""
            }
            page_response = requests.get(page_url, params=page_params).json()

            eol_data = {}
            # Extract Habitat and Reproduction information from data objects
            data_objects = page_response.get('dataObjects', [])
            for obj in data_objects:
                if obj.get('subject') == 'Habitat':
                    eol_data['Elinympäristö (EOL)'] = obj.get('description')
                if obj.get('subject') == 'Reproduction':
                    eol_data['Lisääntyminen (EOL)'] = obj.get('description')

            print("✅ EOL: Ekologiset tiedot haettu.")
            return eol_data
        else:
            print(f"❌ EOL: Ei löytänyt tietoa nimelle: {tieteellinen_nimi}")
            return None
    except Exception as e:
        print(f"❌ EOL haku epäonnistui: {e}")
        return None

def hae_gbif_esiintymat(tieteellinen_nimi):
    """Hakee esiintymistietojen määrän GBIF API:sta."""
    # First, get the GBIF taxon key for the species
    match_url = f"https://api.gbif.org/v1/species/match?name={tieteellinen_nimi}"
    try:
        match_response = requests.get(match_url).json()
        if match_response and match_response.get('usageKey'):
            species_key = match_response['usageKey']

            # Then, count occurrences for that taxon key
            count_url = f"https://api.gbif.org/v1/occurrence/count?taxonKey={species_key}"
            count_response = requests.get(count_url)

            if count_response.status_code == 200:
                occurrence_count = count_response.json()
                print("✅ GBIF: Esiintymistiedot haettu.")
                return {"GBIF esiintymismäärä": occurrence_count}
            else:
                print(f"❌ GBIF esiintymistietojen haku epäonnistui. Status koodi: {count_response.status_code}")
                return None
        else:
            print(f"❌ GBIF ei löytänyt taxon avainta nimelle: {tieteellinen_nimi}")
            return None
    except Exception as e:
        print(f"❌ GBIF esiintymistietojen haku epäonnistui: {e}")
        return None

# Integrate the new function calls into the main program flow
eol_data = hae_eol_data(kasvin_nimi) if API_ASETUKSET.get("eol", True) else None
gbif_occurrence_data = hae_gbif_esiintymat(kasvin_nimi) if API_ASETUKSET.get("gbif_occ", True) else None

if not API_ASETUKSET.get("eol", True):
    print("ℹ️ EOL-haku jätettiin välistä asetuksien perusteella.")
if not API_ASETUKSET.get("gbif_occ", True):
    print("ℹ️ GBIF esiintymismäärä -haku jätettiin välistä asetuksien perusteella.")

❌ EOL haku epäonnistui: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
✅ GBIF: Esiintymistiedot haettu.


In [ ]:
API_ASETUKSET = globals().get("API_ASETUKSET", {})

def hae_wikipedia_tiedot(tieteellinen_nimi, kielet=("fi", "en")):
    """Hakee tiiviin yhteenvedon ja kuvalinkin Wikipedia REST API:sta."""

# Update the DataFrame with new data, handling cases where data was not found
new_data_rows = []

if gbif_occurrence_data:
    new_data_rows.append({"Tieto": "GBIF esiintymismäärä", "Arvo": gbif_occurrence_data.get('GBIF esiintymismäärä')})
else:
    new_data_rows.append({"Tieto": "GBIF esiintymismäärä", "Arvo": "N/A"})

if eol_data:
    new_data_rows.append({"Tieto": "Elinympäristö (EOL)", "Arvo": eol_data.get('Elinympäristö (EOL)', 'N/A')})
    new_data_rows.append({"Tieto": "Lisääntyminen (EOL)", "Arvo": eol_data.get('Lisääntyminen (EOL)', 'N/A')})
else:
    new_data_rows.append({"Tieto": "Elinympäristö (EOL)", "Arvo": "N/A"})
    new_data_rows.append({"Tieto": "Lisääntyminen (EOL)", "Arvo": "N/A"})

if wikipedia_data:
    wiki_summary = wikipedia_data.get('yhteenveto')
    if wiki_summary and len(wiki_summary) > 500:
        wiki_summary = wiki_summary[:497] + "..."
    wiki_language = wikipedia_data.get('kieli', 'fi/en')
    new_data_rows.append({"Tieto": f"Wikipedia yhteenveto ({wiki_language})", "Arvo": wiki_summary or "N/A"})
    if wikipedia_data.get('lyhyt_kuvaus'):
        new_data_rows.append({"Tieto": "Wikipedia lyhyt kuvaus", "Arvo": wikipedia_data.get('lyhyt_kuvaus')})
    if wikipedia_data.get('artikkeli_url'):
        new_data_rows.append({"Tieto": "Wikipedia artikkeli", "Arvo": wikipedia_data.get('artikkeli_url')})
else:
    new_data_rows.append({"Tieto": "Wikipedia yhteenveto", "Arvo": "N/A"})

if inaturalist_data:
    new_data_rows.append({"Tieto": "iNaturalist havaintomäärä", "Arvo": inaturalist_data.get('havaintojen_maara', 'N/A')})
    if inaturalist_data.get('yleisnimi'):
        new_data_rows.append({"Tieto": "iNaturalist yleisnimi", "Arvo": inaturalist_data.get('yleisnimi')})
    if inaturalist_data.get('sivu_url'):
        new_data_rows.append({"Tieto": "iNaturalist sivu", "Arvo": inaturalist_data.get('sivu_url')})
else:
    new_data_rows.append({"Tieto": "iNaturalist havaintomäärä", "Arvo": "N/A"})

if gbif_distribution_data:
    distribution_summary = gbif_distribution_data.get('levinneisyys')
    if not distribution_summary:
        distribution_summary = "Saatavilla, mutta ilman nimettyjä alueita"
    new_data_rows.append({"Tieto": "GBIF levinneisyys (tiivistelmä)", "Arvo": distribution_summary})
    new_data_rows.append({"Tieto": "GBIF levinneisyyslähteiden määrä", "Arvo": gbif_distribution_data.get('lahteet')})
else:
    new_data_rows.append({"Tieto": "GBIF levinneisyys (tiivistelmä)", "Arvo": "N/A"})

kasvi_df = pd.concat([kasvi_df, pd.DataFrame(new_data_rows)], ignore_index=True)

# Display the updated DataFrame
display(Markdown(f"# 🌲 KASVIKORTTI: **{kasvin_nimi}**"))
display(Markdown("***"))
display(kasvi_df.style.hide(axis='index'))

# Redisplay images if they were found previously
display(Markdown(f"\n## 🖼 Kuvat ja Kuvitus"))

if kasvitaulu_url:
    display(Markdown("### 🎨 Kasvitaulu (Wikimedia Commons)"))
    display(Image(url=kasvitaulu_url, width=400))
else:
    display(Markdown("> *Kasvitaulua ei löytynyt Wikimedia Commonsista.*"))

trefle_kuva_url = trefle_data.get('Kuva URL (Trefle)') if trefle_data else None
if trefle_kuva_url:
    display(Markdown("### 📸 Valokuva (Trefle API)"))
    display(Image(url=trefle_kuva_url, width=400))
elif not kasvitaulu_url:
    display(Markdown("> *Valokuvaakaan ei ollut saatavilla Trefle API:sta.*"))

eol_images_cached = globals().get('eol_image_urls')
if eol_images_cached:
    display(Markdown("### 🌿 Kuvat (Encyclopedia of Life)"))
    for img_url in eol_images_cached:
        display(Image(url=img_url, width=400))

def hae_bhl_kuvat(tieteellinen_nimi):
    """Yrittää hakea kasvikuvituksia Biodiversity Heritage Library (BHL) API:sta."""
    base_url = "https://www.biodiversitylibrary.org/api2/httpquery.ashx"
    api_key = globals().get("BHL_API_KEY", "YOUR_BHL_API_KEY")

    if not api_key or api_key == "YOUR_BHL_API_KEY":
        print("❌ BHL: API-avain puuttuu. Lisää BHL_API_KEY muuttuja ennen hakua.")
        return None

    # Step 1: Search for titles containing the scientific name
    search_params = {
        "op": "BookSearch",
        "searchterm": tieteellinen_nimi,
        "apikey": api_key,
        "format": "json"
    }

    try:
        search_response = requests.get(base_url, params=search_params, timeout=15)
        search_response.raise_for_status()
        search_data = search_response.json()
    except Exception as error:
        print(f"❌ BHL: haku epäonnistui ({error}).")
        return None

    if search_data.get("Status") != "ok" or not search_data.get("Result"):
        print(f"❌ BHL: Ei löytynyt hakutuloksia nimelle {tieteellinen_nimi}.")
        return None

    item_id = search_data["Result"][0].get("ItemID")
    if not item_id:
        print(f"❌ BHL: Ei ItemID:tä nimelle {tieteellinen_nimi}.")
        return None

    images_params = {
        "op": "GetItemImages",
        "itemid": item_id,
        "apikey": api_key,
        "format": "json"
    }

    try:
        images_response = requests.get(base_url, params=images_params, timeout=15)
        images_response.raise_for_status()
        images_data = images_response.json()
    except Exception as error:
        print(f"❌ BHL: kuvan haku epäonnistui ({error}).")
        return None

    if images_data.get("Status") != "ok" or not images_data.get("Result"):
        print("❌ BHL: Ei kuvatietoja löytynyt haetusta kohteesta.")
        return None

    bhl_image_urls = []
    for page in images_data.get("Result", []):
        page_id = page.get("PageID")
        if page_id:
            image_url = f"https://www.biodiversitylibrary.org/pageimage/{page_id}"
            bhl_image_urls.append(image_url)
        if len(bhl_image_urls) >= 5:
            break

    if not bhl_image_urls:
        print("❌ BHL: Ei kuvia löytynyt haetusta kohteesta.")
        return None

    print(f"✅ BHL: {len(bhl_image_urls)} kuvaa haettu.")
    return bhl_image_urls


# Call the new BHL image search function
if API_ASETUKSET.get("bhl", False):
    bhl_image_urls = hae_bhl_kuvat(kasvin_nimi)
else:
    print("ℹ️ BHL-kuvahaku jätettiin välistä asetuksien perusteella.")
    bhl_image_urls = None

# Päivitetty esityskerros uusilla tietolähteillä
display(Markdown(f"# 🌲 KASVIKORTTI: **{kasvin_nimi}**"))
display(Markdown("***"))

display(kasvi_df.style.hide(axis='index'))

display(Markdown("\n## 🖼 Kuvat ja Kuvitus"))

images_found = False

if kasvitaulu_url:
    images_found = True
    display(Markdown("### 🎨 Kasvitaulu (Wikimedia Commons)"))
    display(Image(url=kasvitaulu_url, width=400))
elif API_ASETUKSET.get("wikimedia", True):
    display(Markdown("> *Kasvitaulua ei löytynyt Wikimedia Commonsista.*"))

trefle_kuva_url = trefle_data.get('Kuva URL (Trefle)') if trefle_data else None
if trefle_kuva_url:
    images_found = True
    display(Markdown("### 📸 Valokuva (Trefle API)"))
    display(Image(url=trefle_kuva_url, width=400))
elif API_ASETUKSET.get("trefle", True):
    display(Markdown("> *Valokuvaakaan ei ollut saatavilla Trefle API:sta.*"))

wikipedia_image_url = wikipedia_data.get('kuva_url') if wikipedia_data else None
if wikipedia_image_url:
    images_found = True
    display(Markdown("### 📘 Kuva (Wikipedia)"))
    display(Image(url=wikipedia_image_url, width=400))

inaturalist_image_url = inaturalist_data.get('kuva_url') if inaturalist_data else None
if inaturalist_image_url:
    images_found = True
    display(Markdown("### 🌍 Valokuva (iNaturalist)"))
    display(Image(url=inaturalist_image_url, width=400))

if eol_image_urls:
    images_found = True
    display(Markdown("### 🌿 Kuvat (Encyclopedia of Life)"))
    for img_url in eol_image_urls:
        display(Image(url=img_url, width=400))

if bhl_image_urls:
    images_found = True
    display(Markdown("### 📚 Kasvitaulu (Biodiversity Heritage Library)"))
    for img_url in bhl_image_urls:
        display(Image(url=img_url, width=400))

if not images_found:
    display(Markdown("> *Yhtään kuvaa ei ollut saatavilla käytetyistä lähteistä.*"))

link_sections = []
if wikipedia_data and wikipedia_data.get('artikkeli_url'):
    link_sections.append(f"[Wikipedia-artikkeli ({wikipedia_data.get('kieli', 'fi')})]({wikipedia_data['artikkeli_url']})")
if inaturalist_data and inaturalist_data.get('sivu_url'):
    link_sections.append(f"[iNaturalist-sivu]({inaturalist_data['sivu_url']})")

if link_sections:
    display(Markdown("\n".join(link_sections)))

Check if the Gemini API key is available and then use the collected plant data to generate a summary using the Gemini API.



In [ ]:
import google.generativeai as genai

# 1. Check if GEMINI_API_KEY is available
if not GEMINI_API_KEY or GEMINI_API_KEY == "Syötä Gemini API key, AI sisällön generointia varten: ":
    print("❌ Gemini API key puuttuu. AI-sisällön generointi ohitetaan.")
else:
    # 2. Assemble collected data for Gemini API
    plant_summary_input = f"Luo lyhyt, yleistajuinen kuvaus kasvista perustuen seuraaviin tietoihin:\n\n"
    plant_summary_input += f"Tieteellinen nimi: {kasvin_nimi}\n"
    plant_summary_input += f"Suomenkielinen nimi: {laji_nimi}\n"
    if trefle_data and trefle_data.get('Yleisnimi (English)'):
         plant_summary_input += f"Yleisnimi (English): {trefle_data.get('Yleisnimi (English)')}\n"
    if gbif_data:
        plant_summary_input += f"Valtakunta: {gbif_data.get('Valtakunta (Kingdom)')}\n"
        plant_summary_input += f"Heimo: {gbif_data.get('Heimo (Family)')}\n"
        plant_summary_input += f"Suku: {gbif_data.get('Suku (Genus)')}\n"
        plant_summary_input += f"Taksonin tila: {gbif_data.get('Taksonin tila')}\n"
    if trefle_data and trefle_data.get('Kasvumuoto'):
         plant_summary_input += f"Kasvumuoto: {trefle_data.get('Kasvumuoto')}\n"
    if trefle_data and trefle_data.get('pH-alue'):
         plant_summary_input += f"Optimaalinen pH-alue: {trefle_data.get('pH-alue')}\n"
    if gbif_occurrence_data and gbif_occurrence_data.get('GBIF esiintymismäärä') is not None:
         plant_summary_input += f"GBIF esiintymismäärä: {gbif_occurrence_data.get('GBIF esiintymismäärä')}\n"
    if eol_data and eol_data.get('Elinympäristö (EOL)'):
         plant_summary_input += f"Elinympäristö (EOL): {eol_data.get('Elinympäristö (EOL)')}\n"
    if eol_data and eol_data.get('Lisääntyminen (EOL)'):
         plant_summary_input += f"Lisääntyminen (EOL): {eol_data.get('Lisääntyminen (EOL)')}\n"


    # 3. Configure the google.generativeai library
    genai.configure(api_key=GEMINI_API_KEY)

    try:
        # 4. Instantiate the GenerativeModel with a known available model
        # Check the available models to find one that supports generate_content
        list_models_response = genai.list_models()
        available_models = [m.name for m in list_models_response if 'generateContent' in m.supported_generation_methods]

        if not available_models:
            print("❌ Gemini API: No models supporting generateContent available.")
            gemini_description = None
        else:
            # Use the first available model that supports generateContent
            model_name = available_models[0]
            model = genai.GenerativeModel(model_name)
            print(f"✅ Gemini API: Using model '{model_name}' for generation.")

            # 5. Create a detailed prompt in Finnish
            prompt = f"Kirjoita lyhyt, noin 100-150 sanan yleistajuinen kuvaus tästä kasvista suomeksi, perustuen annettuihin tietoihin. Mainitse ainakin tieteellinen nimi, suomenkielinen ja englanninkielinen nimi (jos tiedossa) sekä viittaa sen levinneisyyteen (käyttäen GBIF esiintymämäärää, jos tiedossa).\n\nTiedot:\n{plant_summary_input}"

            # 6. Call the generate_content method on the instantiated model
            response = model.generate_content(prompt)

            # 7. If the API call is successful, extract the generated text
            gemini_description = response.text
            print("✅ Gemini API: Kuvaus generoitu.")

            # 8. Store the generated text (already in gemini_description)

            # 9. Print the generated description in Markdown format
            display(Markdown("## 📝 AI-generoitu kuvaus"))
            display(Markdown(gemini_description))

    except Exception as e:
        # 10. Include error handling
        print(f"❌ Gemini API haku epäonnistui: {e}")
        gemini_description = None # Ensure variable is set even on failure

In [ ]:
# 4.1 Tietojen yhdistäminen ja esittäminen
if gbif_data or trefle_data:

    # Alusta DataFrame tiedoilla
    data_kortti = {
        "Tieto": ["Tieteellinen nimi", "Suomenkielinen nimi", "Valtakunta", "Heimo", "Suku", "Taksonin tila", "Yleisnimi (English)", "Kasvumuoto", "Optimaalinen pH-alue"],
        "Arvo": [
            kasvin_nimi,
            laji_nimi,
            gbif_data.get('Valtakunta (Kingdom)') if gbif_data else "N/A",
            gbif_data.get('Heimo (Family)') if gbif_data else "N/A",
            gbif_data.get('Suku (Genus)') if gbif_data else "N/A",
            gbif_data.get('Taksonin tila') if gbif_data else "N/A",
            trefle_data.get('Yleisnimi (English)') if trefle_data else "N/A",
            trefle_data.get('Kasvumuoto') if trefle_data else "N/A",
            trefle_data.get('pH-alue') if trefle_data else "N/A"
        ]
    }
    kasvi_df = pd.DataFrame(data_kortti)

    # 4.2 Tulostus
    display(Markdown(f"# 🌲 KASVIKORTTI: **{kasvin_nimi}**"))
    display(Markdown(f"***"))

    # Näytä tiedot Pandas-taulukkona
    display(kasvi_df.style.hide(axis='index'))

    # 4.3 Kuvat
    display(Markdown(f"\n## 🖼 Kuvat ja Kuvitus"))

    # Ensin Kasvitaulu (akateeminen kuvitus)
    if kasvitaulu_url:
        display(Markdown("### 🎨 Kasvitaulu (Wikimedia Commons)"))
        display(Image(url=kasvitaulu_url, width=400))
    else:
        display(Markdown("> *Kasvitaulua ei löytynyt Wikimedia Commonsista.*"))

    # Sitten Trefle API:n kuva (jos saatavilla)
    trefle_kuva_url = trefle_data.get('Kuva URL (Trefle)') if trefle_data else None
    if trefle_kuva_url:
        display(Markdown("### 📸 Valokuva (Trefle API)"))
        display(Image(url=trefle_kuva_url, width=400))
    elif not kasvitaulu_url:
        display(Markdown("> *Valokuvaakaan ei ollut saatavilla Trefle API:sta.*"))

else:
    display(Markdown(f"**HUOM:** Tietojen haku kasville **{kasvin_nimi}** epäonnistui useassa API:ssa. Tarkista nimi!"))

NameError: name 'laji_nimi' is not defined